In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_log = pd.read_csv('tianchi/data_format1/user_log_format1.csv', sep=',')
data_info = pd.read_csv('tianchi/data_format1/user_info_format1.csv', sep=',')
data_train = pd.read_csv('tianchi/data_format2/train_format2.csv', sep=',')

In [4]:
data_test = pd.read_csv('tianchi/data_format2/test_format2.csv', sep=',')

In [5]:
data_train.head()

,user_id,age_range,gender,merchant_id,label,activity_log
0,34176,6.0,0.0,944,-1,408895:1505:7370:1107:0
1,34176,6.0,0.0,412,-1,17235:1604:4396:0818:0#954723:1604:4396:0818:0...
2,34176,6.0,0.0,1945,-1,231901:662:2758:0818:0#231901:662:2758:0818:0#...
3,34176,6.0,0.0,4752,-1,174142:821:6938:1027:0
4,34176,6.0,0.0,643,-1,716371:1505:968:1024:3


In [6]:
data_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [7]:
data_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [8]:
data1_train = pd.read_csv('tianchi/data_format1/train_format1.csv', sep=',')

In [9]:
data1_test = pd.read_csv('tianchi/data_format1/test_format1.csv', sep=',')

In [10]:
data1_test['train'] = 0
data1_train['train'] = 1

In [11]:
matrix = pd.concat([data1_train, data1_test], ignore_index=True, sort=False)

In [12]:
matrix.head()

,user_id,merchant_id,label,train,prob
0,34176,3906,0.0,1,NaN
1,34176,121,0.0,1,NaN
2,34176,4356,1.0,1,NaN
3,34176,2217,0.0,1,NaN
4,230784,4818,0.0,1,NaN


In [13]:
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(data_info, on='user_id', how='left')

In [14]:
matrix.head()

,user_id,merchant_id,label,train,age_range,gender
0,34176,3906,0.0,1,6.0,0.0
1,34176,121,0.0,1,6.0,0.0
2,34176,4356,1.0,1,6.0,0.0
3,34176,2217,0.0,1,6.0,0.0
4,230784,4818,0.0,1,0.0,0.0


In [15]:
data_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

In [16]:
data_log.head()

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [17]:
data_log['action_type'].unique()

array([0, 2, 3, 1])

In [18]:
data_log['time_stamp'].dtype

dtype('int64')

In [19]:
matrix.isnull().sum()

user_id             0
merchant_id         0
label          261477
train               0
age_range        2578
gender           7545
dtype: int64

In [20]:
matrix.gender.unique()

array([ 0.,  1.,  2., nan])

In [21]:
matrix.age_range.unique()

array([ 6.,  0.,  4.,  5.,  3.,  2.,  7., nan,  8.,  1.])

In [22]:
matrix.age_range = matrix.age_range.replace(0., np.nan)

In [23]:
matrix.age_range.unique()

array([ 6., nan,  4.,  5.,  3.,  2.,  7.,  8.,  1.])

In [24]:
matrix.isnull().sum()

user_id             0
merchant_id         0
label          261477
train               0
age_range      114245
gender           7545
dtype: int64

In [25]:
matrix[matrix['train'] == 1].isnull().sum()

user_id            0
merchant_id        0
label              0
train              0
age_range      57062
gender          3711
dtype: int64

In [26]:
groups = data_log.groupby('user_id')

In [27]:
groups['item_id'].agg([('u2', 'nunique')]).reset_index()

,user_id,u2
0,1,12
1,2,43
2,3,45
3,4,28
4,5,87
...,...,...
424165,424166,48
424166,424167,15
424167,424168,160
424168,424169,176


In [28]:
matrix.gender.nunique()

3

In [29]:
groups.size()

user_id
1          33
2          63
3          68
4          50
5         173
         ... 
424166     90
424167     35
424168    223
424169    297
424170     40
Length: 424170, dtype: int64

In [30]:
###特征处理，增加用户对商品的点击次数、商品的购买次数、浏览次数等特征
temp = groups.size().reset_index().rename(columns={0:'user_click_count'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups[['item_id', 'cat_id', 'brand_id', 'merchant_id']].nunique().reset_index().rename(columns={
    'item_id': 'user_item_count',
    'cat_id': 'user_cat_count',
    'brand_id': 'user_brand_count',
    'merchant_id': 'user_merchant_count'
})
matrix = matrix.merge(temp, on='user_id', how='left')

temp = groups['time_stamp'].agg([('start_time', 'min'), ('end_time', 'max')]).reset_index()

temp['start_end_time'] = (temp['end_time'] - temp['start_time'])/3600

matrix = matrix.merge(temp[['user_id', 'start_end_time']], on='user_id', how='left')


In [31]:
temp = groups[['item_id', 'cat_id', 'brand_id']].nunique().reset_index().rename(columns={
    'item_id': 'user_item_count',
    'cat_id': 'user_cat_count',
    'brand_id': 'user_brand_count'
})

In [32]:
groups['action_type'].value_counts().unstack()

action_type,0,1,2,3
user_id,,,,
1,27.0,NaN,6.0,NaN
2,47.0,NaN,14.0,2.0
3,63.0,NaN,4.0,1.0
4,49.0,NaN,1.0,NaN
5,150.0,NaN,13.0,10.0
...,...,...,...,...
424166,79.0,NaN,11.0,NaN
424167,28.0,NaN,6.0,1.0
424168,216.0,NaN,6.0,1.0


In [33]:
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'click', 1:'add_item', 2: 'buy', 3: 'collect'})

In [34]:
matrix = matrix.merge(temp, on='user_id', how='left')

In [35]:
temp = groups['time_stamp'].agg([('start_time', 'min'), ('end_time', 'max')]).reset_index()

temp['start_end_time'] = (temp['end_time'] - temp['start_time'])/3600

matrix = matrix.merge(temp[['user_id', 'start_end_time']], on='user_id', how='left')

In [36]:
groups_merchant = data_log.groupby(['merchant_id'])
temp_mer = groups_merchant.size().reset_index().rename(columns={0: 'merchant_count'})
matrix = matrix.merge(temp_mer, on='merchant_id', how='left')

temp_mer = groups_merchant[['item_id', 'cat_id', 'brand_id', 'user_id']].nunique().reset_index().rename(columns={
    'item_id': 'merchant_item_count',
    'cat_id': 'merchant_cat_count',
    'brand_id': 'merchant_brand_count',
    'user_id': 'merchant_user_count'
})

matrix = matrix.merge(temp_mer, on='merchant_id', how='left')

temp_mer = groups_merchant['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'merchant_click', 1: 'merchant_add', 2: 'merchant_buy', 3: 'merchant_collct'})

matrix = matrix.merge(temp_mer, on='merchant_id', how='left')

In [37]:
###特征组合
groups_user_mer = data_log.groupby(['user_id', 'merchant_id'])

temp_um = groups_user_mer.size().reset_index().rename(columns={0: 'user_merchant_count'})

matrix = matrix.merge(temp_um, on=['user_id', 'merchant_id'], how='left')
temp_um = groups_user_mer['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'item_id':'user_merchant_item',
    'cat_id':'user_merchant_cat',
    'brand_id':'user_merchant_brand'
})
matrix = matrix.merge(temp_um, on=['user_id', 'merchant_id'], how='left')
temp_um = groups_user_mer['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'user_merchant_click', 1: 'user_merchant_add', 2: 'user_merchant_buy', 3: 'user_merchant_collect'})
matrix = matrix.merge(temp_um, on=['user_id', 'merchant_id'], how='left')
temp_um = groups_user_mer['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp_um['user_merchant_time'] = (temp_um['first'] - temp_um['last']) / 3600
matrix = matrix.merge(temp_um[['user_merchant_time', 'user_id', 'merchant_id']], on=['user_id', 'merchant_id'], how='left')


In [38]:
matrix['buy_rents'] = matrix.buy / matrix.click 
matrix['merchant_rents'] = matrix.merchant_buy / matrix.merchant_click
matrix['user_merchant_rents'] = matrix.user_merchant_buy / matrix.user_merchant_click

In [39]:
matrix.shape

(522341, 38)

In [40]:
matrix.isnull().sum() / len(matrix.index)

user_id                  0.000000
merchant_id              0.000000
label                    0.500587
train                    0.000000
age_range                0.218717
gender                   0.014445
user_click_count         0.000000
user_item_count          0.000000
user_cat_count           0.000000
user_brand_count         0.000000
user_merchant_count_x    0.000000
start_end_time_x         0.000000
click                    0.000689
add_item                 0.926908
buy                      0.000000
collect                  0.435505
start_end_time_y         0.000000
merchant_count           0.000000
merchant_item_count      0.000000
merchant_cat_count       0.000000
merchant_brand_count     0.000000
merchant_user_count      0.000000
merchant_click           0.000000
merchant_add             0.007757
merchant_buy             0.000000
merchant_collct          0.000000
user_merchant_count_y    0.000000
user_merchant_item       0.000000
user_merchant_cat        0.000000
user_merchant_

In [41]:
matrix.drop(['add_item', 'user_merchant_add', 'user_merchant_collect'], axis=1, inplace=True)

In [42]:
data_train = matrix[matrix['train'] ==1]

In [43]:
data_test = matrix[matrix['train'] ==0]

In [44]:
###one_hot编码
temp_oh = pd.get_dummies(matrix, columns=['age_range', 'gender'])

In [45]:
temp_oh.head()

,user_id,merchant_id,label,train,user_click_count,user_item_count,user_cat_count,user_brand_count,user_merchant_count_x,start_end_time_x,...,age_range_2.0,age_range_3.0,age_range_4.0,age_range_5.0,age_range_6.0,age_range_7.0,age_range_8.0,gender_0.0,gender_1.0,gender_2.0
0,34176,3906,0.0,1,451,256,45,108,109,0.163889,...,0,0,0,0,1,0,0,1,0,0
1,34176,121,0.0,1,451,256,45,108,109,0.163889,...,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1.0,1,451,256,45,108,109,0.163889,...,0,0,0,0,1,0,0,1,0,0
3,34176,2217,0.0,1,451,256,45,108,109,0.163889,...,0,0,0,0,1,0,0,1,0,0
4,230784,4818,0.0,1,54,31,17,19,20,0.141667,...,0,0,0,0,0,0,0,1,0,0


In [46]:
matrix.isnull().sum()

user_id                       0
merchant_id                   0
label                    261477
train                         0
age_range                114245
gender                     7545
user_click_count              0
user_item_count               0
user_cat_count                0
user_brand_count              0
user_merchant_count_x         0
start_end_time_x              0
click                       360
buy                           0
collect                  227482
start_end_time_y              0
merchant_count                0
merchant_item_count           0
merchant_cat_count            0
merchant_brand_count          0
merchant_user_count           0
merchant_click                0
merchant_add               4052
merchant_buy                  0
merchant_collct               0
user_merchant_count_y         0
user_merchant_item            0
user_merchant_cat             0
user_merchant_brand           0
user_merchant_click       59408
user_merchant_buy             0
user_mer

In [47]:
matrix.age_range.fillna(0, inplace=True)

In [48]:
matrix.gender.fillna(2, inplace=True)

In [49]:
matrix_fill = matrix.drop(['label'], axis=1)

In [50]:
matrix_fill.fillna(0, inplace=True)

In [51]:
matrix_fill['label'] = matrix['label']

In [52]:
matrix_fill.isnull().sum()

user_id                       0
merchant_id                   0
train                         0
age_range                     0
gender                        0
user_click_count              0
user_item_count               0
user_cat_count                0
user_brand_count              0
user_merchant_count_x         0
start_end_time_x              0
click                         0
buy                           0
collect                       0
start_end_time_y              0
merchant_count                0
merchant_item_count           0
merchant_cat_count            0
merchant_brand_count          0
merchant_user_count           0
merchant_click                0
merchant_add                  0
merchant_buy                  0
merchant_collct               0
user_merchant_count_y         0
user_merchant_item            0
user_merchant_cat             0
user_merchant_brand           0
user_merchant_click           0
user_merchant_buy             0
user_merchant_time            0
buy_rent

In [53]:
matrix_fill = pd.get_dummies(matrix_fill, columns=['age_range', 'gender'])
train_data = matrix_fill[matrix_fill['train'] == 1]
test_data = matrix_fill[matrix_fill['train'] == 0]

In [54]:
train_data.head()

,user_id,merchant_id,train,user_click_count,user_item_count,user_cat_count,user_brand_count,user_merchant_count_x,start_end_time_x,click,...,age_range_2.0,age_range_3.0,age_range_4.0,age_range_5.0,age_range_6.0,age_range_7.0,age_range_8.0,gender_0.0,gender_1.0,gender_2.0
0,34176,3906,1,451,256,45,108,109,0.163889,410.0,...,0,0,0,0,1,0,0,1,0,0
1,34176,121,1,451,256,45,108,109,0.163889,410.0,...,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1,451,256,45,108,109,0.163889,410.0,...,0,0,0,0,1,0,0,1,0,0
3,34176,2217,1,451,256,45,108,109,0.163889,410.0,...,0,0,0,0,1,0,0,1,0,0
4,230784,4818,1,54,31,17,19,20,0.141667,47.0,...,0,0,0,0,0,0,0,1,0,0


In [55]:
test_data.drop(['label', 'train'], axis=1, inplace=True)
train_data.drop(['train'], axis=1, inplace=True)

In [56]:
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

In [57]:
##多种模型比较
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [58]:
###交叉验证
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
models = [
         LogisticRegression(),
         DecisionTreeRegressor(),
         Lasso(),
         Ridge(),
         SVC(),
         RandomForestRegressor(),
         GradientBoostingRegressor(),
         LinearRegression()]

result = dict()
for model in models:
    model_name = str(model).split('(')[0]
    scores = cross_val_score(model, X=train_X, y=train_y, verbose=0, cv=5, scoring=make_scorer(mean_absolute_error))
    
    result[model_name] = scores
    
    print(model_name + ' is finished')

LogisticRegression is finished
DecisionTreeRegressor is finished
Lasso is finished
Ridge is finished
SVC is finished
RandomForestRegressor is finished
GradientBoostingRegressor is finished
LinearRegression is finished


In [59]:
result = pd.DataFrame(result)
result.index = ['cv' + str(i) for i in range(1, 6)]
result.T

,cv1,cv2,cv3,cv4,cv5
LogisticRegression,0.061143,0.061143,0.061162,0.061162,0.061144
DecisionTreeRegressor,0.123014,0.124586,0.125927,0.124317,0.123074
Lasso,0.113981,0.114640,0.115277,0.113790,0.114411
Ridge,0.112398,0.112740,0.113596,0.112274,0.112519
SVC,0.061143,0.061143,0.061162,0.061162,0.061144
RandomForestRegressor,0.124137,0.124865,0.125552,0.125421,0.124809
GradientBoostingRegressor,0.111800,0.112241,0.113080,0.111878,0.112068
LinearRegression,0.112397,0.112640,0.113632,0.112274,0.112518


In [60]:
from sklearn.model_selection import GridSearchCV

def get_best_model_and_accuracy(model, params, X, y):
    grid = GridSearchCV(model, ##要搜索的模型
                       params, ##要尝试的参数
                       error_score=0.)  ##如果报错，结果是0
    grid.fit(X, y) #拟合模型和参数
    # 性能指标
    print('best accuracy: {}'.format(grid.best_score_))
    # 得到最佳准确率的最佳参数
    print('best parameters:{}'.format(grid.best_params_))
    # 拟合的平均时间(秒)
    print('average time to fit (s): {}'.format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    ## 预测的平均时间
    # 从该指标可以看出模型在真实世界的性能
    print("Average Time to Score (s):{}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

In [61]:
## 为网格搜索设置变量
## 先设置机器学习模型的参数

#逻辑回归
lr_params = {'C':[1e-1, 1e0, 1e1, 1e2], 'penalty':['l1', 'l2']}

#决策树
tree_params = {'max_depth':[None, 1,3,5,7]}

#随机森林
forest_params = {'n_estimators':[10,50,100], 'max_depth':[None, 1,3,5,7]}

gbdt_params = {'n_estimators':[10,50,100], 'max_depth':[None, 1,3,5,7]}

In [62]:
get_best_model_and_accuracy(RandomForestRegressor(), forest_params, train_X, train_y)

best accuracy: 0.02204798566808086
best parameters:{'max_depth': 7, 'n_estimators': 100}
average time to fit (s): 69.293
Average Time to Score (s):0.235


In [63]:
model = RandomForestRegressor(n_estimators=100, max_depth=7)
model_fini = model.fit(train_X, train_y)

In [64]:
result = model_fini.predict(test_data)

In [93]:
sub = pd.DataFrame()
sub['prob'] = result

In [94]:
sub

,prob
0,0.064197
1,0.101180
2,0.050375
3,0.037348
4,0.048482
...,...
261472,0.035231
261473,0.046299
261474,0.107251
261475,0.038792


In [67]:
sub[sub['prob']>=0.5]

,prob


In [68]:
model = LinearRegression()
model = model.fit(train_X, train_y)

In [69]:
result = model.predict(test_data)

In [70]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(
    oob_score=True, 
    n_jobs=-1, 
    n_estimators=1000, 
    max_depth=10, 
    max_features='sqrt')

model = rf_clf.fit(train_X, train_y)

In [71]:
sub.prob = sub.prob

In [72]:
sub

,prob
0,0
1,0
2,0
3,0
4,0
...,...
261472,0
261473,0
261474,0
261475,0


In [73]:
submission = pd.read_csv('tianchi/data_format1/test_format1.csv', sep=',')

In [74]:
submission['prob'] = sub.prob

In [75]:
submission.to_csv('submission.csv', index=False)

In [76]:
from sklearn.model_selection import train_test_split
train_X, valid_X, trian_y, valid_y = train_test_split(train_X, train_y, test_size=.3)

In [84]:
train_X

,user_id,merchant_id,user_click_count,user_item_count,user_cat_count,user_brand_count,user_merchant_count_x,start_end_time_x,click,buy,...,age_range_2.0,age_range_3.0,age_range_4.0,age_range_5.0,age_range_6.0,age_range_7.0,age_range_8.0,gender_0.0,gender_1.0,gender_2.0
41385,334074,4871,30,17,10,10,12,0.111389,22.0,7.0,...,0,0,0,0,1,0,0,0,1,0
14040,260265,4662,115,64,23,35,37,0.111111,107.0,8.0,...,0,1,0,0,0,0,0,0,1,0
169622,118900,598,50,34,20,24,25,0.165833,37.0,12.0,...,0,0,0,0,0,0,0,1,0,0
225816,91160,1892,311,170,53,64,61,0.164444,230.0,21.0,...,0,1,0,0,0,0,0,1,0,0
73411,252504,2469,611,317,99,79,73,0.166111,525.0,64.0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260311,388478,3828,51,26,14,11,11,0.140278,42.0,9.0,...,0,0,0,0,0,0,0,1,0,0
99765,367270,35,113,62,24,38,44,0.141389,109.0,3.0,...,1,0,0,0,0,0,0,0,1,0
94316,22165,2808,26,8,2,5,8,0.023611,22.0,4.0,...,0,0,0,0,0,0,0,1,0,0
84663,273273,1703,104,64,29,25,26,0.161389,95.0,9.0,...,0,1,0,0,0,0,0,0,1,0


In [83]:
import xgboost

In [82]:
!brew install libomp

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
abseil                     geph2                      oil
archiver                   ghc@8.8                    openfast
asuka                      ghz                        openlibm
autodiff                   git-bug                    openstackclient
azcopy                     git-remote-codecommit      pass-git-helper
bcoin                      git-trim                   pfetch
beancount                  gmailctl                   polynote
bnfc                       go-jsonnet                 publish
bootloadhid                go@1.13                    qp
cbmc                       gobo                       qrcp
ccheck                     gocloc                     quill
cdk8s                      graphql-cli                rbtools
cdo                        guile@2                    rcm
chezmoi                    hcxtools                   redo
claws-mail                 hdf5-mpi    


==> Downloading https://homebrew.bintray.com/bottles/libomp-10.0.0.catalina.bott
==> Downloading from https://akamai.bintray.com/0e/0ea757dbea7bf12141ef1d209d2f3
######################################################################## 100.0%
==> Pouring libomp-10.0.0.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/libomp/10.0.0: 9 files, 1.3MB
==> `brew cleanup` has not been run in 30 days, running now...
Removing: /Users/lvsongke/Library/Caches/Homebrew/aom--1.0.0.catalina.bottle.1.tar.gz... (4.1MB)
Removing: /Users/lvsongke/Library/Caches/Homebrew/autoconf--2.69.catalina.bottle.4.tar.gz... (871.6KB)
Removing: /Users/lvsongke/Library/Caches/Homebrew/automake--1.16.1_1.catalina.bottle.tar.gz... (945.3KB)
Removing: /Users/lvsongke/Library/Caches/Homebrew/cairo--1.16.0_2.catalina.bottle.tar.gz... (1.6MB)
Removing: /Users/lvsongke/Library/Caches/Homebrew/carthage--0.34.0.catalina.bottle.tar.gz... (1.7MB)
Removing: /Users/lvsongke/Library/Caches/Homebrew/ceres-solver--1.14.0_9.catalina.bottle

Pruned 0 symbolic links and 1 directories from /usr/local


In [87]:
model = xgboost.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42
    
)

model.fit(
    train_X, 
    trian_y,
    eval_metric='auc',
    eval_set=[(train_X, trian_y), (valid_X, valid_y)],
    verbose=True,
    early_stopping_rounds=10
)


[0]	validation_0-auc:0.63296	validation_1-auc:0.61690
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.64902	validation_1-auc:0.63148
[2]	validation_0-auc:0.65989	validation_1-auc:0.63969
[3]	validation_0-auc:0.66427	validation_1-auc:0.64461
[4]	validation_0-auc:0.66455	validation_1-auc:0.64438
[5]	validation_0-auc:0.66728	validation_1-auc:0.64443
[6]	validation_0-auc:0.66938	validation_1-auc:0.64613
[7]	validation_0-auc:0.67057	validation_1-auc:0.64749
[8]	validation_0-auc:0.67179	validation_1-auc:0.64785
[9]	validation_0-auc:0.67293	validation_1-auc:0.64869
[10]	validation_0-auc:0.67330	validation_1-auc:0.64849
[11]	validation_0-auc:0.67411	validation_1-auc:0.64884
[12]	validation_0-auc:0.67545	validation_1-auc:0.65046
[13]	validation_0-auc:0.67746	validation_1-auc:0.65090
[14]	validation_0-auc:0.67819	validation_1-auc:0.65176
[15]	validation_0-auc:0.6796

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=300, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [95]:
# prob = model.predict_proba(test_data)
submission['prob'] = sub.prob
# submission.dorp(['origin'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

In [89]:
submission.head()

,user_id,merchant_id,prob
0,163968,4605,0.049807
1,360576,1581,0.099645
2,98688,1964,0.058759
3,98688,3645,0.036599
4,295296,3361,0.047039


In [90]:
submission.prob.describe()

count    261477.000000
mean          0.061300
std           0.045683
min           0.007120
25%           0.032511
50%           0.047405
75%           0.073609
max           0.525518
Name: prob, dtype: float64

In [2]:
train_X

NameError: name 'train_X' is not defined